In [1]:
import os
import re
from numba import jit, cuda
import random
import tkinter as tk
from tkinter.filedialog import askopenfilename
import numpy as np
from numpy import asarray
from scipy import misc
import imageio
%matplotlib inline 
from matplotlib import image as matimg
import matplotlib.pyplot as plt
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array
from deeplab import DeepLabV3Plus, ASPP
from resnet50 import ResNet50
import tensorflow as tf
import cv2
from tqdm import tqdm
import os
import random
from PIL import Image
from glob import glob
import pickle
import time
import json
from tensorflow.python.keras.utils.all_utils import multi_gpu_model
from tensorflow.python.keras.utils.all_utils import Sequence
from moviepy.editor import VideoFileClip, ImageSequenceClip
from keras.applications.imagenet_utils import preprocess_input

print('Tensorflow', tf.__version__)

Tensorflow 2.4.1


In [2]:
h, w = 800, 1600
with open('cityscapes_dict.pkl', 'rb') as f:
    id_to_color = pickle.load(f)['color_map']

In [3]:
id_to_color[8] = (128, 64,128) # sidewalk to road
id_to_color[6] = (128, 64,128) # ground to road
id_to_color[12] = (70, 70, 70) # wall to building
id_to_color[13] = (70, 70, 70) # fence to building
id_to_color[14] = (70, 70, 70) # guard rail to building

In [4]:
model = DeepLabV3Plus(h, w, 34)
model.load_weights('last_epoch.h5')

*** Building DeepLabv3Plus Network ***
*** Output_Shape => (None, 800, 1600, 34) ***


In [ ]:
# def pipeline(image, video=True, return_seg=False, fname='', folder='', downsampling=1):
#     global b
#     alpha = 0.5
#     dims = image.shape
#     image = cv2.resize(image, (w, h))
#     x = image.copy()
#     z = model.predict(preprocess_input(np.expand_dims(x, axis=0)))
    
#     z = np.squeeze(z)
#     y = np.argmax(z, axis=2)
    
#     img_color = image.copy()   
#     for i in np.unique(y):
#         if i in id_to_color:
#             img_color[y==i] = id_to_color[i]
#     disp = img_color.copy()
#     if video:
#         cv2.addWeighted(image, alpha, img_color, 1-alpha, 0, img_color)        
#         return img_color
#     if return_seg:
#         return img_color/255.
#     else:
#         cv2.addWeighted(image, alpha, img_color, 1-alpha, 0, img_color)      
# #         plt.figure(figsize=(20, 10))
# #         out = np.concatenate([image/255, img_color/255, disp/255], axis=1)
        
# #         plt.imshow(img_color/255.0)
# #         plt.imshow(out)
#         if not os.path.exists('Output'):
#             os.makedirs('Output')
#         return cv2.imwrite(f'Output/{folder}/{fname}',  cv2.cvtColor(img_color, cv2.COLOR_RGB2BGR))    

In [63]:
class SegmentationPipeline():
    def __init__(self, video=True, return_seg=False, folder='', fname='{}.jpg', downsample=1, interp='linear'):
        self.video = video
        self.return_seg = return_seg
        self.folder = folder
        self.downsample = downsample
        self.prev_z = None
        self.index = 0
        self.fname = fname
        
        if interp != 'linear':
            raise NotImplementedError
    
    def pipe(self, image, overlays=None):
        alpha = 0.5
        dims = image.shape
        image = cv2.resize(image, (w, h))
        x = image.copy()
        curr_z = model.predict(preprocess_input(np.expand_dims(x, axis=0)))
        curr_z = np.squeeze(curr_z)
        
        for i in range(self.downsample):
#             print(self.index)
            if self.index == 0 and i < self.downsample - 1:
                continue
            
            if self.downsample > 1 and i < self.downsample - 1:
                curr_weight = (i+1)/self.downsample
                prev_weight = 1. - curr_weight

                interp_z = prev_weight * self.prev_z + curr_weight * curr_z
            else:
                interp_z = curr_z
            
            y = np.argmax(interp_z, axis=2)
            
            img_color = image.copy()   
            for j in np.unique(y):
                if j in id_to_color:
                    img_color[y==j] = id_to_color[j]

            if self.video:
                cv2.addWeighted(image, alpha, img_color, 1-alpha, 0, img_color)        
                out = img_color
            else:
                if self.return_seg:
                    out = img_color/255.
                else:
                    if overlays is not None:
                        if len(overlays) > 1:
                            overlay = overlays[i]  
                        else:
                            overlay = overlays[0]
                        overlay = cv2.resize(overlay, (w, h))
                    else:
                        overlay = image

                    cv2.addWeighted(overlay, alpha, img_color, 1-alpha, 0, img_color)

                    if not os.path.exists('Output'):
                        os.makedirs('Output')
                        
                    thefilename = self.fname.format(self.index)
#                     print(thefilename)
                    out = cv2.imwrite(f'Output/{self.folder}/{thefilename}',  cv2.cvtColor(img_color, cv2.COLOR_RGB2BGR))    
                    
            self.index += 1
    
        self.prev_z = curr_z
        
        
        return out

In [36]:
root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

file_path = askopenfilename(parent=root,title='Select .txt') 
os.chdir(os.path.dirname(os.path.realpath(file_path)))
directory_path = os.getcwd()
image_dir = sorted(glob(directory_path + "/*.jpg",recursive=False))
print("File path is: " + file_path)
print("Working diretory is: " + directory_path)
print(f'{len(image_dir)} frames found')

File path is: E:/chengyong/gitproject/DeepLabv3plus_tf2-4-1/imgs/test_0001.jpg
Working diretory is: E:\chengyong\gitproject\DeepLabv3plus_tf2-4-1\imgs
100 frames found


In [37]:
image_dir = sorted(image_dir)

In [59]:
video_fps = 30
segmentation_fps = 10
assert video_fps % segmentation_fps == 0
downsample = video_fps//segmentation_fps

In [64]:
pipeline = SegmentationPipeline(
    video=False, 
    return_seg=False, 
    folder='', 
    fname='waterfeature_{}.jpg', 
    downsample=downsample, 
    interp='linear'
)

overlays = []
for i in tqdm(range(len(image_dir)), desc = "Prediction Progress"):
    try:
        test = load_img(image_dir[i])
        test = img_to_array(test)
        
        overlays.append(test)
        
        if i % downsample == 0:
            segmap = pipeline.pipe(
                test,
                overlays
            )
            overlays = []
        if segmap == False:
            break
    except Exception as e:
        print(str(e))

Prediction Progress: 100%|███████████████████████████████████████████████████████████| 100/100 [01:15<00:00,  1.33it/s]


In [61]:
def numericalMergeIntoVideo(directory_path):
    dirFiles = []
    for filename in glob(directory_path + "/*.jpg",recursive=False):
        dirFiles.append(filename)

    dirFiles.sort(key=lambda f: int(re.sub('\D', '', f)))

    img_array = []
    for filename in tqdm(dirFiles, desc="Reading Progress"):
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)

    video = cv2.VideoWriter(directory_path + '/30fps.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 30, size)

    for i in tqdm(range(len(img_array)), desc = "Video Merging Progress"):
        video.write(img_array[i])

    cv2.destroyAllWindows()
    video.release()

In [62]:
numericalMergeIntoVideo(directory_path + "/Output")

Video Merging Progress: 100%|███████████████████████████████████████████████████████| 100/100 [00:00<00:00, 109.12it/s]


In [ ]:
pwd